## HierarchyNet Example (Base: VGG-16)

Salma Taoufiq

This notebook provides an example implementation of the HierarchyNet architecture on a VGG-16 base model.

### Setting the stage

In [ ]:
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from keras.utils import get_file
from keras import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten, BatchNormalization, Lambda, Softmax, concatenate
from keras.optimizers import SGD
from keras import backend
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns


In [ ]:
# Working with 3-channel (RGB) 224x224 images for the VGG16 model
INPUT_SHAPE = (224, 224, 3)
COARSE_CATEGORIES = ['Religious', 'Residential', 'Commercial', 'Business']
FINE_CATEGORIES = ['Church', 'Mosque', 'Synagogue', 'BuddhistTemple', 'House', 'ApartmentBuilding', 'Mall', 'Store', 'Restaurant', 'OfficeBuilding']
NB_COARSE_CLASSES = 4
NB_FINE_CLASSES = 10

In [ ]:
#Definining the mapping fine:coarse
fine_to_coarse = {0:0, 1:0, 2:0, 3:0, 4:1, 5:1, 6:2, 7:2, 8:2, 9:3}

#### Reading the dataset:

In [ ]:
photos = np.load('./FinalDatasetPhotos_array.npy', allow_pickle=True)
fine_labels = np.load('./FinalDatasetLabels_array.npy', allow_pickle=True)

In [ ]:
from keras.layers import Lambda

def crop(dimension, start, end):
    '''
    Crops a Tensor from start to end on the specified dimension
    '''
    def func(x):
        slices = [slice(None)] * x.ndim
        slices[dimension] = slice(start, end)
        return x[tuple(slices)]
    return Lambda(func)


In [ ]:
def proba_product(tensors):
    '''
    Returns the element-wise product of two tensors.
    '''
    return(tf.multiply(tensors[0], tensors[1]))

In [ ]:
# Define the custom multiplicative layer
custom_layer = Lambda(proba_product)

In [ ]:
# Importing the weights of the pretrained VGG16
weights_file = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
weights = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5', weights_file, cache_subdir='models')

In [ ]:
#!nvidia-smi

In [ ]:
loss_w1 = 0.3 #tf.Variable(initial_value=0.3, dtype="float32", name="coarse_loss_weight")
loss_w2 = 0.7 #tf.Variable(initial_value=0.7, dtype="float32", name="fine_loss_weight")

In [ ]:
#loss_w1: Loss weight for the coarse branch
#loss_w2: Loss weight for the fine branch

class ModifyLossWeights(keras.callbacks.Callback):
    def __init__(self, loss_w1, loss_w2):
        self.loss_w1 = loss_w1
        self.loss_w2 = loss_w2
    def on_epoch_end(self, epoch, logs={}):
        if epoch == 15:
            keras.backend.set_value(self.loss_w1, 0.5)
            keras.backend.set_value(self.loss_w2, 0.5)
        #if epoch == 35:
          #keras.backend.set_value(self.loss_w1, 0)
          #keras.backend.set_value(self.loss_w2, 1)

In [ ]:
#np.random.seed(0)

# Model input
model_input = Input(shape=INPUT_SHAPE, name='image_input')
# Block 1
model = Conv2D(64, (3,3), padding='same', activation='relu', name='block1_conv1')(model_input)
model = BatchNormalization()(model)
model = Conv2D(64, (3,3), padding='same', activation='relu', name='block1_conv2')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block1_pool')(model)
# Block 2
model = Conv2D(128, (3,3), padding='same', activation='relu', name='block2_conv1')(model)
model = BatchNormalization()(model)
model = Conv2D(128, (3,3), padding='same', activation='relu', name='block2_conv2')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block2_pool')(model)
# Block 3
model = Conv2D(256, (3,3), padding='same', activation='relu', name='block3_conv1')(model)
model = BatchNormalization()(model)
model = Conv2D(256, (3,3), padding='same', activation='relu', name='block3_conv2')(model)
model = BatchNormalization()(model)
model = Conv2D(256, (3,3), padding='same', activation='relu', name='block3_conv3')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block3_pool')(model)
# Block 4
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block4_conv1')(model)
model = BatchNormalization()(model)
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block4_conv2')(model)
model = BatchNormalization()(model)
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block4_conv3')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block4_pool')(model)
# Block 5
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block5_conv1')(model)
model = BatchNormalization()(model)
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block5_conv2')(model)
model = BatchNormalization()(model)
model = Conv2D(512, (3,3), padding='same', activation='relu', name='block5_conv3')(model)
model = BatchNormalization()(model)
model = MaxPooling2D((2,2), strides=(2,2), name='block5_pool')(model)

flatten = Flatten(name='c_flatten')(model)
branch = Dense(256, activation='relu', name='c_dense1')(flatten)
branch = BatchNormalization()(branch)
branch = Dropout(0.5)(branch)
branch = Dense(128, activation='relu', name='c_dense2')(branch)
branch = BatchNormalization()(branch)
branch = Dropout(0.45)(branch)

coarse_branch = Dense(NB_COARSE_CLASSES)(branch)

coarse_1 = crop(1, 0, 1)(coarse_branch)
coarse_2 = crop(1, 1, 2)(coarse_branch)
coarse_3 = crop(1, 2, 3)(coarse_branch)
coarse_4 = crop(1, 3, 4)(coarse_branch)

coarse_pred = Softmax(name='coarse_prediction')(coarse_branch)

subclasses = Dense(NB_FINE_CLASSES)(branch)
sub_1 = crop(1, 0, 4)(subclasses)
sub_2 = crop(1, 4, 6)(subclasses)
sub_3 = crop(1, 6, 9)(subclasses)
sub_4 = crop(1, 9, 10)(subclasses)

fine_1= custom_layer([coarse_1, sub_1])
fine_2= custom_layer([coarse_2, sub_2])
fine_3= custom_layer([coarse_3, sub_3])
fine_4 = custom_layer([coarse_4, sub_4])

merged_fine = concatenate([fine_1, fine_2, fine_3, fine_4])
fine_pred = Softmax(name='fine_prediction')(merged_fine)

model = Model(inputs=model_input, outputs=[coarse_pred, fine_pred])
model.load_weights(weights, by_name=True)
opt = SGD(learning_rate=0.0001, momentum=0.9, clipvalue=0.5)
bcnn_losses = {"coarse_prediction": "categorical_crossentropy", "fine_prediction": "categorical_crossentropy"}
#{"coarse_prediction": "accuracy", "fine_prediction": "accuracy"}
initial_loss_weights = {"coarse_prediction": loss_w1, "fine_prediction": loss_w2}
model.compile(optimizer=opt, loss=bcnn_losses, metrics=["accuracy", "accuracy"], loss_weights=initial_loss_weights)

In [ ]:
model.summary()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(photos, fine_labels, test_size=0.2,
                                               random_state = np.random.randint(1,356, 1)[0])
X_train = X_train/255
X_test = X_test/255

# Creating empty matrices for the coarse labels of the train and test sets with the appropriate shapes
y_coarse_train = np.zeros(y_train.shape[0]).astype("float32")
y_coarse_test = np.zeros(y_test.shape[0]).astype("float32")

# Filling up the matrices accordingly by putting 1 in the column corresponding to the correct coarse class
for i in range(y_coarse_train.shape[0]):
    y_coarse_train[i] = fine_to_coarse[y_train[i]]
for i in range(y_coarse_test.shape[0]):
    y_coarse_test[i] = fine_to_coarse[y_test[i]]

In [ ]:
#Preparing the callback:
loss_w_modif = ModifyLossWeights(loss_w1, loss_w2)

In [ ]:
y_test = tf.keras.utils.to_categorical(y_test, num_classes = NB_FINE_CLASSES)
y_train = tf.keras.utils.to_categorical(y_train, num_classes = NB_FINE_CLASSES)
y_coarse_test = tf.keras.utils.to_categorical(y_coarse_test, num_classes = NB_COARSE_CLASSES)
y_coarse_train = tf.keras.utils.to_categorical(y_coarse_train, num_classes = NB_COARSE_CLASSES)

#callbacks=[loss_w_modif]
trained_model = model.fit(X_train, [y_coarse_train, y_train], batch_size=50, epochs=50, verbose=1, validation_split=0.1)
print("Val Score: ", model.evaluate(X_test, [y_coarse_test, y_test]))

In [ ]:
trained_model.history

In [ ]:
fig = plt.plot(trained_model.history["coarse_prediction_accuracy"],label = "Training - Coarse", color='blue')
plt.plot(trained_model.history["val_coarse_prediction_accuracy"],label = "Validation - Coarse", color='green')
plt.plot(trained_model.history["fine_prediction_accuracy"],label = "Training - Fine", color='brown')
plt.plot(trained_model.history["val_fine_prediction_accuracy"],label = "Validation - Fine", color='orange')
plt.legend(loc='lower right')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Accuracy per epoch")
plt.show()

In [ ]:
'''
plt.plot(architecture_model.history["coarse_prediction_loss"],label = "Training - Coarse", color='blue')
plt.plot(architecture_model.history["val_coarse_prediction_loss"],label = "Testing - Coarse", color='green')
plt.plot(architecture_model.history["fine_prediction_loss"],label = "Training - Fine", color='brown')
plt.plot(architecture_model.history["val_fine_prediction_loss"],label = "Testing - Fine", color='orange')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()
'''

In [ ]:
coarse_preds = model.predict(X_test)[0]
fine_preds = model.predict(X_test)[1]